In [ ]:
import pandas as pd
import duckdb as ddb
import numpy as np

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
con = ddb.connect("./kalam_hydropower.db")

## Analysis and Feature Engineering
Now that the data has been aggregated, it is easier to work with in pandas and visualise using matplotlib and seaborn. The inention of this section is to:
- Produce features that are intuitive - i.e. hypothesize on what could support forecasting
- Visualise the feature signals to understand how the data varies with the target signal over time
- Check the feature covariation with the target data (daily kwh) to get a sense of which features to try

We start by examining the high-level features of the daily hydropower production data.

In [ ]:
dhp_df = con.sql("""
    select
        *
    from prepared.daily_hydropower_production
""").to_df()

dhp_df.head()

In [ ]:
dhp_df.info()

In [ ]:
aggregated_dhp_df = dhp_df[["date", "kwh"]].groupby("date", as_index=False).sum()

In [ ]:
aggregated_dhp_df.plot(x="date", y="kwh", figsize=(18, 5), title="Daily Power Production")

In [ ]:
ss_df = con.sql("select * from raw.sample_submission").to_df()

ss_df.head()

In [ ]:
ss_df[["date", "source"]] = ss_df['ID'].str.split('_', n=1, expand=True)

ss_df.head()

In [ ]:
print(f"Number of unique sources in actual data: {len(dhp_df['source'].unique())}")
print(f"Number of unique sources in submission data: {len(ss_df['source'].unique())}")

In [ ]:
# These are devices with weak signals which we can exclude from the analysis - these were mentioned in the samplesubmission notebook
devices_to_drop = ["3", "5", "11", "14", "15", "17", "24", "25", "27", "33", "4", "9"]

In [ ]:
to_forecast_df = dhp_df[~(dhp_df["consumer_device"].isin(devices_to_drop))].copy()

In [ ]:
# Pivot the data such that all the different device, user combinations become independent columns at the same length
to_forecast_pivotted_df = to_forecast_df.pivot(columns="source", index="date", values="kwh").copy()

# Not all of the signals have the same length, in this case I simply fill missing/non-existant values with a 0
to_forecast_pivotted_df.fillna(0, inplace=True)

to_forecast_pivotted_df.head()

In [ ]:
# Now I melt the dataframe back into a row-based structure, but I am guarenteed that all signals are the same length
to_forecast_extended_df = to_forecast_pivotted_df.melt(value_vars=list(to_forecast_pivotted_df.columns), var_name="source", value_name="kwh", ignore_index=False)

to_forecast_extended_df.reset_index(inplace=True, drop=False)

to_forecast_extended_df.head()

In [ ]:
# I create a dataframe with the sum of kwh across all user/device combinations for easy visualisation (i.e. one series instead of over 400)
power_df = to_forecast_extended_df[["date", "kwh"]].groupby("date", as_index=False).sum()

power_df.head()

In [ ]:
# Now I pull the daily climate data into a dataframe for feature engineering
climate_df = con.sql("select * from prepared.daily_climate").to_df()

climate_df.head()

In [ ]:
# Features taken from an example notebook - these seem intuitive as power production likely depends on rainfall and temperature (melting of snow)
climate_df["temp_dew_diff"] = climate_df["avg_temperature"] - climate_df["avg_dewpoint_temperature"]
climate_df["wind_speed"] = (climate_df["avg_u_wind_component"]**2 + climate_df["avg_v_wind_component"]**2)**0.5
climate_df["precip_snow_ratio"] = climate_df["total_precipitation"] / (climate_df["total_snowfall"] + 1e-6)
climate_df["precip_minus_snow"] = climate_df["total_precipitation"] - climate_df["total_snowfall"]

# Some more 'out there' features to try - these were basically some brainstorm features I thought might work
climate_df["rolling_precip_7d"] = climate_df["total_precipitation"].rolling(7).sum()
climate_df["rolling_snow_30d"] = climate_df["total_snowfall"].rolling(30).sum()
climate_df["melt_potential"] = climate_df["avg_temperature"] * climate_df["avg_snow_cover_perc"]
climate_df["wind_variability_3d"] = climate_df["wind_speed"].rolling(3).std()

In [ ]:
# Temporal features - features based on time/seasonality
climate_df["dayofyear"] = climate_df["date"].dt.dayofyear
climate_df["month"] = climate_df["date"].dt.month
climate_df["sin_doy"] = np.sin(2 * np.pi * climate_df["dayofyear"] / 365)
climate_df["cos_doy"] = np.cos(2 * np.pi * climate_df["dayofyear"] / 365)

In [ ]:
all_data_df = pd.merge(climate_df, power_df, on="date", how="left")

all_data_df.head()

In [ ]:
# The features and the target data are all on different scales, and I want to observe the shape of the signals,
# so for this I produce a scaled set of data to visualise
scaler = StandardScaler()

numeric_columns = all_data_df.select_dtypes("number").columns

scaled_data_df = all_data_df.copy()
scaled_data_df[numeric_columns] = scaler.fit_transform(all_data_df[numeric_columns])

In [ ]:
# Here I define the feature columns I want to plot against kwh
feature_columns = list(all_data_df.columns)

feature_columns.remove("date")
feature_columns.remove("kwh")

In [ ]:
# Define the kWh components and feature components
kwh_components = ['kwh']
features = feature_columns

# Create subplots: one row per feature
fig, axes = plt.subplots(len(features), len(kwh_components), figsize=(18, 50), sharex=True)
fig.suptitle('Comparison of kWh Components with Weather Features (Scaled)', fontsize=16)

for i, feature in enumerate(features):
    ax = axes[i]
    ax.plot(scaled_data_df['date'], scaled_data_df[feature], label=feature)
    ax.plot(scaled_data_df['date'], scaled_data_df["kwh"], label="kwh", linestyle='--')
    ax.set_ylabel(feature)
    if i == len(features) - 1:
        ax.set_xlabel('Date')
    if i == 0:
        ax.set_title("Total kwh")
    ax.legend(loc='upper right')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

What is clear from these sub-plots is the following:
- There is a significant 'downtime' period from 2024-01 to 2024-07, no power is being produced between those times
- Precipitation minus snow (only rainfall) coincides with power production
- Snowfall does not seem to have an impact in 2024 - i.e. high snowfall does not imply high power production
- Temperature rises in 2024 seem to coincide with power production
- There is clearly a seasonality in power production, and temporal features reflect that, sin_doy, dayofyear and month seem to be good candidates to reflect this

Now, given that this is a time series problem it may not be that the date-aligned features at some lag may be better predictors. I want to understand correlation over various lags of the features.

In [ ]:
# Ensure that the data is ordered appropriately for the correlation check
all_data_df.sort_values(by="date", inplace=True)
all_data_df.head()

In [ ]:
# Define lag periods (in days)
lags = {
    '0_lag': 0,
    '1_day': 1,
    '1_week': 7,
    '1_month': 30,
    '2_month': 60,
    '1_quarter': 90
}

# Store correlations for each lag
lagged_corrs = pd.DataFrame(index=features)

for label, lag_days in lags.items():
    df_lagged = all_data_df[all_data_df["date"] >= pd.Timestamp("2024-01-01")].copy()
    for feature in features:
        df_lagged[feature + '_lag'] = df_lagged[feature].shift(lag_days)
    df_lagged = df_lagged.dropna(subset=[f + '_lag' for f in features])
    
    corr_series = df_lagged[[f + '_lag' for f in features] + ['kwh']].corr()['kwh']
    # Drop 'kwh' itself and rename the index to remove '_lag'
    corr_series = corr_series.drop('kwh')
    corr_series.index = [i.replace('_lag', '') for i in corr_series.index]
    
    lagged_corrs[label] = corr_series

# Plot the results
plt.figure(figsize=(12, 12))
sns.heatmap(lagged_corrs, annot=True, cmap='coolwarm', center=0)
plt.title('Lagged Correlation of Weather Features with avg_kwh')
plt.ylabel('Feature')
plt.xlabel('Lag Period')
plt.show()


The following features at a lag of 30 days (~1month) seem to be good candidates
- Temperature/dewpoint temperature
- Wind speed
- Precip minus snow (rainfall)
- Precip to snow ratio

Now some of the features have strong correlations (like those involving snow) but looking at the signal plots, these could be spurrious relationships and don't make intuitive sense to use.

In [ ]:
# The features are stored for future use
try:
    con.sql("""
        create table prepared.daily_features as
            select * from climate_df
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

In [ ]:
con.close()